# Tutorial 02: Neuron Morphology

**Author:** Alexander Bates  
**Python Version with navis**

---

## Introduction

The purpose of this tutorial is to examine neuron morphology using the **navis** Python library.

We can visualize neurons, compare their morphological similarity with NBLAST, and analyze their spatial properties.

This tutorial demonstrates:
- Loading neuron skeletons (.swc files) from GCS with navis
- 3D visualization with plotly
- Morphometric analysis (cable length, branch points, etc.)
- Visualizing neurons with neuropil meshes
- NBLAST morphological similarity analysis
- Template brain transformations with flybrains

---

## About navis

[**navis**](https://navis.readthedocs.io/) is a Python library for analyzing and visualizing neuron morphology and connectivity. It provides:

- Support for multiple neuron representations (skeletons, meshes, point clouds)
- Fast NBLAST implementation for morphological comparison
- Template brain transformations via [**flybrains**](https://github.com/navis-org/navis-flybrains)
- Multiple 3D visualization backends (plotly, octarine, k3d)
- Seamless integration with other neuroinformatics tools

navis is the Python equivalent of the R natverse ecosystem.

---

## Setup and Configuration

In [ ]:
# Dataset configuration# Options: "banc_746", "fafb_783", "manc_121", "hemibrain_121", "malecns_09"DATASET = "banc_746"DATASET_ID = "banc_746_id"# We'll focus on a specific neuropil subset to keep examples manageableSUBSET_NAME = "front_leg"NEUROPIL_PATTERN = r"^LegNp\(T1\)|T1|^ProNM-T1|^LNp_T3"# Data location - can be GCS bucket or local pathDATA_PATH = "gs://sjcabs_2025_data"# Detect if using GCS or local pathUSE_GCS = DATA_PATH.startswith("gs://")# Image output directoryimport osIMG_DIR = "images/tutorial_02"os.makedirs(IMG_DIR, exist_ok=True)print(f"Dataset: {DATASET}")print(f"Subset: {SUBSET_NAME}")print(f"Data location: {DATA_PATH}")print(f"Using GCS: {USE_GCS}")print(f"Images will be saved to: {IMG_DIR}")

## Import Packages

We'll use:
- **navis**: Neuron analysis and visualization
- **pandas**: Data manipulation
- **pyarrow**: Reading Feather files
- **gcsfs**: Google Cloud Storage access
- **plotly**: Interactive 3D visualizations
- **trimesh**: 3D mesh processing

**Installation:**
```bash
pip install navis[all] flybrains gcsfs plotly kaleido trimesh
```

In [ ]:
# Import all common packages and helper functions
import sys
sys.path.insert(0, '.')
from setup_helpers import *

print(f"navis version: {navis.__version__}")
print(f"pandas version: {pd.__version__}")

## Setup GCS Access

**Authentication required:** Before running with GCS, authenticate:

```bash
gcloud auth application-default login
```

In [ ]:
# Setup GCS filesystem if needed
if USE_GCS:
    print("Setting up Google Cloud Storage access...")
    gcs = gcsfs.GCSFileSystem(token='google_default')
    print("✓ GCS filesystem initialized")
else:
    gcs = None
    print("Using local filesystem")

## Helper Functions

In [ ]:
def construct_path(data_root, dataset, file_type="meta", space_suffix=None):
    """
    Construct file paths for dataset files.
    
    Parameters
    ----------
    data_root : str
        Root data directory (can be gs:// or local path)
    dataset : str
        Dataset name with version (e.g., "banc_746")
    file_type : str
        Type of file: "meta", "synapses", "skeletons"
    space_suffix : str, optional
        Space name for skeletons (defaults to native space)
    
    Returns
    -------
    str
        Full path to the file
    """
    dataset_name = dataset.split("_")[0]
    
    extensions = {
        "meta": ".feather",
        "synapses": ".parquet",
        "skeletons": ""  # No extension - it's a directory
    }
    
    if file_type not in extensions:
        raise ValueError(f"Unknown file_type: {file_type}")
    
    extension = extensions[file_type]
    
    if file_type == "skeletons":
        # Skeleton directories don't include version number
        if space_suffix is None:
            space_suffix = f"{dataset_name}_space"
        
        # BANC uses l2 skeletons
        if dataset_name == "banc":
            filename = f"{dataset_name}_{space_suffix}_l2_swc{extension}"
        else:
            filename = f"{dataset_name}_{space_suffix}_swc{extension}"
    else:
        filename = f"{dataset}_{file_type}{extension}"
    
    full_path = f"{data_root}/{dataset_name}/{filename}"
    return full_path


def read_feather_gcs(path, gcs_fs=None):
    """Read Feather file from GCS or local path."""
    if path.startswith("gs://"):
        if gcs_fs is None:
            raise ValueError("gcs_fs required for GCS paths")
        
        print(f"Reading from GCS: {path}")
        gcs_path = path.replace("gs://", "")
        
        with gcs_fs.open(gcs_path, 'rb') as f:
            df = feather.read_feather(f)
        
        print(f"✓ Loaded {len(df):,} rows")
        return df
    else:
        print(f"Reading from local path: {path}")
        df = pd.read_feather(path)
        print(f"✓ Loaded {len(df):,} rows")
        return df


def read_swc_from_gcs(gcs_fs, swc_path):
    """
    Read a single SWC file from GCS using navis.
    
    Parameters
    ----------
    gcs_fs : gcsfs.GCSFileSystem
        GCS filesystem object
    swc_path : str
        GCS path to SWC file (without gs:// prefix)
    
    Returns
    -------
    navis.TreeNeuron
        Loaded neuron
    """
    # Read file content from GCS
    with gcs_fs.open(swc_path, 'rb') as f:
        content = f.read()
    
    # Create a file-like object from bytes
    swc_file = io.BytesIO(content)
    
    # Read with navis
    neuron = navis.read_swc(swc_file)
    
    return neuron


def batch_read_swc_from_gcs(gcs_fs, directory, filenames, show_progress=True):
    """
    Batch read multiple SWC files from GCS.
    
    Parameters
    ----------
    gcs_fs : gcsfs.GCSFileSystem
        GCS filesystem object
    directory : str
        GCS directory path (without gs:// prefix)
    filenames : list of str
        List of SWC filenames to read
    show_progress : bool
        Whether to show progress bar
    
    Returns
    -------
    navis.NeuronList
        List of loaded neurons
    """
    neurons = []
    
    iterator = tqdm(filenames) if show_progress else filenames
    
    for filename in iterator:
        swc_path = f"{directory}/{filename}"
        try:
            neuron = read_swc_from_gcs(gcs_fs, swc_path)
            neurons.append(neuron)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            continue
    
    return navis.NeuronList(neurons)


def read_obj_from_gcs(gcs_fs, obj_path):
    """
    Read OBJ mesh file from GCS.
    
    Parameters
    ----------
    gcs_fs : gcsfs.GCSFileSystem
        GCS filesystem object
    obj_path : str
        GCS path to OBJ file (without gs:// prefix)
    
    Returns
    -------
    trimesh.Trimesh
        Loaded mesh
    """
    with gcs_fs.open(obj_path, 'rb') as f:
        content = f.read()
    
    # Create temporary file for trimesh
    with tempfile.NamedTemporaryFile(suffix='.obj', delete=False) as tmp:
        tmp.write(content)
        tmp_path = tmp.name
    
    try:
        mesh = trimesh.load_mesh(tmp_path)
    finally:
        os.unlink(tmp_path)
    
    return mesh


print("✓ Helper functions defined")

## Setup File Paths

In [ ]:
# Construct paths
meta_path = construct_path(DATA_PATH, DATASET, "meta")
skeletons_path = construct_path(DATA_PATH, DATASET, "skeletons")

# Extract base dataset name
dataset_base = DATASET.split("_")[0]

print("File paths:")
print(f"  Metadata: {meta_path}")
print(f"  Skeletons: {skeletons_path}")

---

## Read Meta Data

In [ ]:
# Load metadata
meta = read_feather_gcs(meta_path, gcs_fs=gcs)

print(f"\nTotal neurons: {len(meta):,}")
meta.head()

### Explore the Dataset

Let's examine the available neurons in this dataset:

In [ ]:
# For this tutorial, we'll work with DNg12 neurons
# Instead of filtering by neuropil (which requires subset files),
# we'll filter directly by cell type when we need specific neurons

print(f"Total neurons in dataset: {len(meta):,}")
print(f"\nWe'll focus on DNg12 neurons for morphology analysis")

# No pre-filtering needed - we'll select neurons by cell_type as needed
meta.head()

### Examine Cell Types

What are the top cell types in this dataset?

In [ ]:
# Count by cell type
cell_type_counts = meta['cell_type'].value_counts().head(15)

print("Top 15 cell types in dataset:")
print(cell_type_counts)

# Create bar plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=cell_type_counts.index,
    y=cell_type_counts.values,
    marker_color='steelblue',
    text=cell_type_counts.values,
    textposition='outside'
))

fig.update_layout(
    title=f"Top Cell Types: {DATASET}",
    xaxis_title="Cell Type",
    yaxis_title="Count",
    template="plotly_white",
    height=500,
    xaxis_tickangle=-45
)

fig.show()

---

## Read Neuron Skeletons (.swc files)

Neuron skeletons are stored as SWC files in the GCS bucket. We'll use navis to read them.

### Load a Single Neuron

Let's start by loading one DNg12 neuron:

In [ ]:
# Filter for DNg12 neurons from full metadata
meta_dng12 = meta[
    meta['cell_type'].str.contains('DNg12', case=False, na=False)
]

if len(meta_dng12) > 0:
    selected_id = meta_dng12[DATASET_ID].iloc[0]
    print(f"Found {len(meta_dng12)} DNg12 neurons")
    print(f"Selected neuron ID: {selected_id}")
    
    # Construct path to SWC file
    swc_filename = f"{selected_id}.swc"
    
    # Read neuron from GCS
    if USE_GCS:
        gcs_skeleton_path = skeletons_path.replace("gs://", "")
        print(f"Reading from GCS: {gcs_skeleton_path}/{swc_filename}")
        neuron = read_swc_from_gcs(gcs, f"{gcs_skeleton_path}/{swc_filename}")
    else:
        neuron = navis.read_swc(f"{skeletons_path}/{swc_filename}")
    
    print(f"\n✓ Loaded neuron with {neuron.n_nodes:,} nodes")
    print(f"  Cable length: {neuron.cable_length:,.0f} nm")
    print(f"  Number of branches: {neuron.n_branches}")
    print(f"  Number of leaf nodes: {neuron.n_leafs}")
else:
    print("No DNg12 neurons found in this dataset")

### 3D Visualization with Plotly

navis makes it easy to visualize neurons in 3D with plotly backend:

In [ ]:
if len(meta_dng12) > 0:
    # Convert from nm to µm for better scale
    neuron_um = neuron / 1000
    neuron_um.units = 'µm'  # Update units metadata
    
    # Plot with plotly
    # Note: navis.plot3d with plotly backend shows the plot automatically
    navis.plot3d(
        neuron_um,
        backend='plotly',
        color='darkblue',
        width=1200,
        height=800,
        title=f"DNg12 Neuron: {selected_id}"
    )

---

## Read Neuropil Meshes

Neuropil meshes are stored as OBJ files. We can load them and visualize neurons in their anatomical context.

In [ ]:
# Construct path to neuropil meshes
obj_path = f"{DATA_PATH}/{dataset_base}/obj/neuropils"
print(f"Neuropil meshes path: {obj_path}")

# List available OBJ files
if USE_GCS:
    gcs_obj_path = obj_path.replace("gs://", "")
    obj_files = gcs.ls(gcs_obj_path)
    obj_files = [f"gs://{f}" for f in obj_files if f.endswith('.obj')]
else:
    import glob
    obj_files = glob.glob(f"{obj_path}/*.obj")

print(f"\nFound {len(obj_files)} OBJ files")
if obj_files:
    print("Example files:")
    for f in obj_files[:3]:
        print(f"  {f}")

### Load and Visualize Multiple Neurons with Neuropil Context

Let's load VNC and leg neuropil meshes and visualize multiple DNg12 neurons in their anatomical context, each in a different color:

In [ ]:
if len(meta_dng12) > 0 and obj_files:
    # Find VNC and leg neuropil meshes
    vnc_meshes = [f for f in obj_files if 'vnc' in f.lower()]
    leg_meshes = [f for f in obj_files if re.search(NEUROPIL_PATTERN, f, re.IGNORECASE)]
    
    meshes_to_load = []
    if vnc_meshes:
        meshes_to_load.append(vnc_meshes[0])
    if leg_meshes:
        meshes_to_load.append(leg_meshes[0])
    
    if meshes_to_load:
        print(f"Loading {len(meshes_to_load)} mesh(es)...")
        
        loaded_meshes = []
        for mesh_file in meshes_to_load:
            print(f"  {mesh_file}")
            if USE_GCS:
                gcs_mesh_path = mesh_file.replace("gs://", "")
                mesh = read_obj_from_gcs(gcs, gcs_mesh_path)
            else:
                mesh = trimesh.load_mesh(mesh_file)
            
            # Convert to µm to match neuron
            mesh.vertices = mesh.vertices / 1000
            loaded_meshes.append(mesh)
        
        print("\n✓ Meshes loaded")
        
        # Load first 5 DNg12 neurons for visualization
        n_vis = min(5, len(meta_dng12))
        vis_ids = meta_dng12[DATASET_ID].iloc[:n_vis].values
        
        print(f"\nLoading {n_vis} DNg12 neurons for visualization...")
        vis_filenames = [f"{nid}.swc" for nid in vis_ids]
        
        if USE_GCS:
            gcs_skeleton_path = skeletons_path.replace("gs://", "")
            vis_neurons = batch_read_swc_from_gcs(gcs, gcs_skeleton_path, vis_filenames, show_progress=False)
        else:
            vis_neurons = []
            for fname in vis_filenames:
                try:
                    n = navis.read_swc(f"{skeletons_path}/{fname}")
                    vis_neurons.append(n)
                except Exception as e:
                    print(f"Error reading {fname}: {e}")
            vis_neurons = navis.NeuronList(vis_neurons)
        
        # Convert to µm
        vis_neurons_um = vis_neurons / 1000
        
        print(f"✓ Loaded {len(vis_neurons_um)} neurons")
        
        # Create visualization with plotly
        fig = go.Figure()
        
        # Add meshes
        mesh_colors = ['lightgrey', 'lightblue']
        mesh_alphas = [0.1, 0.3]
        mesh_names = ['VNC', 'Leg Neuropil']
        
        for i, mesh in enumerate(loaded_meshes):
            fig.add_trace(go.Mesh3d(
                x=mesh.vertices[:, 0],
                y=mesh.vertices[:, 1],
                z=mesh.vertices[:, 2],
                i=mesh.faces[:, 0],
                j=mesh.faces[:, 1],
                k=mesh.faces[:, 2],
                color=mesh_colors[i % len(mesh_colors)],
                opacity=mesh_alphas[i % len(mesh_alphas)],
                name=mesh_names[i % len(mesh_names)],
                hoverinfo='name'
            ))
        
        # Add neurons in different colors
        neuron_colors = ['darkblue', 'darkred', 'darkgreen', 'darkorange', 'purple']
        
        for i, neuron in enumerate(vis_neurons_um):
            neuron_trace = neuron.nodes
            fig.add_trace(go.Scatter3d(
                x=neuron_trace['x'],
                y=neuron_trace['y'],
                z=neuron_trace['z'],
                mode='lines',
                line=dict(color=neuron_colors[i % len(neuron_colors)], width=3),
                name=f'DNg12 Neuron {i+1}',
                hoverinfo='name'
            ))
        
        fig.update_layout(
            title=f"{len(vis_neurons_um)} DNg12 Neurons in Neuropil Context: {DATASET}",
            scene=dict(
                xaxis_title='X (µm)',
                yaxis_title='Y (µm)',
                zaxis_title='Z (µm)',
                aspectmode='data'
            ),
            width=1200,
            height=800
        )
        
        fig.show()
    else:
        print("No matching neuropil meshes found")
else:
    print("Skipping neuropil visualization (no DNg12 neurons or meshes available)")

---

## Co-plotting Neurons Across Datasets

One powerful feature of our data organization is that we provide neuron skeletons in both native space AND in BANC space. This enables easy co-visualization of neurons from different datasets.

Let's load DNg12 neurons from the **maleCNS** dataset (which are already transformed to BANC space) and co-plot them with our BANC DNg12 neurons:

In [ ]:
# Switch to maleCNS dataset
dataset_male = "malecns_09"
dataset_male_id = "malecns_09_id"

# Read maleCNS meta data
meta_male_path = construct_path(DATA_PATH, dataset_male, "meta")
meta_male_full = read_feather_gcs(meta_male_path, gcs_fs=gcs)

# Filter for DNg12 neurons in maleCNS
meta_male_dng12 = meta_male_full[
    meta_male_full['cell_type'].str.contains('DNg12', case=False, na=False)
]

print(f"Found {len(meta_male_dng12)} DNg12 neurons in maleCNS")

if len(meta_male_dng12) > 0:
    # Construct path to maleCNS skeletons in BANC space
    skeletons_male_banc_path = construct_path(
        DATA_PATH, 
        dataset_male, 
        "skeletons",
        space_suffix="banc_space"
    )
    
    # Read maleCNS neurons (already in BANC space)
    male_ids = meta_male_dng12[dataset_male_id].values
    print(f"Reading {len(male_ids)} maleCNS neurons from BANC space...")
    
    male_filenames = [f"{nid}.swc" for nid in male_ids]
    
    if USE_GCS:
        gcs_male_path = skeletons_male_banc_path.replace("gs://", "")
        neurons_male_banc = batch_read_swc_from_gcs(
            gcs, 
            gcs_male_path, 
            male_filenames, 
            show_progress=False
        )
    else:
        neurons_male_banc = navis.NeuronList([
            navis.read_swc(f"{skeletons_male_banc_path}/{fname}")
            for fname in male_filenames
        ])
    
    # Convert to microns
    neurons_male_banc_um = neurons_male_banc / 1000
    
    print(f"✓ Loaded {len(neurons_male_banc_um)} maleCNS neurons")
    
    # Co-plot BANC and maleCNS neurons
    if len(meta_dng12) > 0 and 'vis_neurons_um' in locals():
        fig = go.Figure()
        
        # Add VNC and brain meshes if available
        if 'loaded_meshes' in locals():
            for i, mesh in enumerate(loaded_meshes):
                mesh_colors_plot = ['lightgrey', 'lightgrey']
                mesh_alphas_plot = [0.1, 0.1]
                mesh_names_plot = ['VNC', 'Brain']
                
                fig.add_trace(go.Mesh3d(
                    x=mesh.vertices[:, 0],
                    y=mesh.vertices[:, 1],
                    z=mesh.vertices[:, 2],
                    i=mesh.faces[:, 0],
                    j=mesh.faces[:, 1],
                    k=mesh.faces[:, 2],
                    color=mesh_colors_plot[i],
                    opacity=mesh_alphas_plot[i],
                    name=mesh_names_plot[i],
                    hoverinfo='name'
                ))
        
        # Plot BANC DNg12 neurons in navy
        for i, neuron in enumerate(vis_neurons_um[:3]):  # Limit to 3 for clarity
            neuron_trace = neuron.nodes
            fig.add_trace(go.Scatter3d(
                x=neuron_trace['x'],
                y=neuron_trace['y'],
                z=neuron_trace['z'],
                mode='lines',
                line=dict(color='navy', width=3),
                name=f'BANC DNg12 {i+1}',
                hoverinfo='name'
            ))
        
        # Plot maleCNS DNg12 neurons in red
        for i, neuron in enumerate(neurons_male_banc_um[:3]):  # Limit to 3 for clarity
            neuron_trace = neuron.nodes
            fig.add_trace(go.Scatter3d(
                x=neuron_trace['x'],
                y=neuron_trace['y'],
                z=neuron_trace['z'],
                mode='lines',
                line=dict(color='red', width=3),
                name=f'maleCNS DNg12 {i+1}',
                hoverinfo='name'
            ))
        
        fig.update_layout(
            title="DNg12 Neurons: BANC (navy) vs maleCNS (red) in BANC Space",
            scene=dict(
                xaxis_title='X (µm)',
                yaxis_title='Y (µm)',
                zaxis_title='Z (µm)',
                aspectmode='data'
            ),
            width=1200,
            height=800
        )
        
        fig.show()
    else:
        print("Skipping co-plot (BANC neurons not loaded)")
else:
    print("No DNg12 neurons found in maleCNS")

---

## Load Multiple Neurons for NBLAST

Let's load neurons from the ventral nerve cord for morphological comparison. Following the R tutorial approach, we'll filter for intrinsic VNC neurons restricted to a single leg neuromere, and take one example per cell type for computational efficiency:

In [ ]:
# Filter for intrinsic VNC neurons in single leg neuromere
# (Same filtering approach as R tutorial for consistency)
meta_vnc = meta[
    (meta['super_class'] == 'ventral_nerve_cord_intrinsic') &
    (meta['cell_class'] == 'single_leg_neuromere')
].copy()

print(f"Found {len(meta_vnc)} VNC intrinsic neurons in single leg neuromere")

# Take one neuron per cell type for speed (as done in R tutorial)
# Note: This is just for tutorial performance, not usually done in real analysis
meta_vnc_sampled = meta_vnc.drop_duplicates(subset='cell_type', keep='first')

print(f"Selected {len(meta_vnc_sampled)} neurons (one per cell type)")
print(f"\nTop cell types:")
print(meta_vnc_sampled['cell_type'].value_counts().head(10))

# Get neuron IDs
sampled_ids = meta_vnc_sampled[DATASET_ID].values

if len(sampled_ids) > 0:
    print(f"\nLoading {len(sampled_ids)} neurons for NBLAST analysis...")
    
    # Prepare filenames
    swc_filenames = [f"{nid}.swc" for nid in sampled_ids]
    
    # Batch read from GCS
    if USE_GCS:
        gcs_skeleton_path = skeletons_path.replace("gs://", "")
        neurons = batch_read_swc_from_gcs(gcs, gcs_skeleton_path, swc_filenames, show_progress=True)
    else:
        # Local reading
        neurons = []
        for fname in tqdm(swc_filenames):
            try:
                n = navis.read_swc(f"{skeletons_path}/{fname}")
                neurons.append(n)
            except Exception as e:
                print(f"Error reading {fname}: {e}")
        neurons = navis.NeuronList(neurons)
    
    print(f"\n✓ Loaded {len(neurons)} neurons successfully")
else:
    print("No neurons found matching the filter criteria")
    neurons = navis.NeuronList([])

### Morphometric Analysis

Calculate morphological properties for all neurons:

In [ ]:
# Convert to microns
neurons_um = neurons / 1000

# Extract morphological properties
morpho_df = pd.DataFrame({
    'neuron_id': sampled_ids[:len(neurons_um)],
    'cell_type': meta_vnc_sampled['cell_type'].values[:len(neurons_um)],
    'cable_length_um': neurons_um.cable_length,
    'n_nodes': [n.n_nodes for n in neurons_um],
    'n_branches': [n.n_branches for n in neurons_um],
    'n_leafs': [n.n_leafs for n in neurons_um]
})

print("\nMorphological Summary:")
print(morpho_df.describe())

morpho_df.head(10)

### Visualize Morphological Distributions

In [ ]:
# Create subplots for different metrics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Cable Length', 'Number of Nodes', 'Number of Branches', 'Number of Leaf Nodes'),
    specs=[[{"type": "histogram"}, {"type": "histogram"}],
           [{"type": "histogram"}, {"type": "histogram"}]]
)

# Cable length
fig.add_trace(
    go.Histogram(x=morpho_df['cable_length_um'], nbinsx=20, marker_color='steelblue', name='Cable Length'),
    row=1, col=1
)

# Number of nodes
fig.add_trace(
    go.Histogram(x=morpho_df['n_nodes'], nbinsx=20, marker_color='seagreen', name='Nodes'),
    row=1, col=2
)

# Number of branches
fig.add_trace(
    go.Histogram(x=morpho_df['n_branches'], nbinsx=20, marker_color='coral', name='Branches'),
    row=2, col=1
)

# Number of leafs
fig.add_trace(
    go.Histogram(x=morpho_df['n_leafs'], nbinsx=20, marker_color='mediumpurple', name='Leaf Nodes'),
    row=2, col=2
)

fig.update_xaxes(title_text="Cable Length (µm)", row=1, col=1)
fig.update_xaxes(title_text="Number of Nodes", row=1, col=2)
fig.update_xaxes(title_text="Number of Branches", row=2, col=1)
fig.update_xaxes(title_text="Number of Leaf Nodes", row=2, col=2)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Count", row=2, col=2)

fig.update_layout(
    title_text=f"Morphological Distributions: {DATASET} - VNC Intrinsic Neurons",
    showlegend=False,
    height=800,
    template="plotly_white"
)

fig.show()

---

## NBLAST Morphological Similarity

**NBLAST** (Neuron BLAST) is an algorithm for comparing neuron morphology based on local geometry. It works by:

1. Converting neurons to point clouds with tangent vectors (dotprops)
2. For each point in query neuron, finding nearest point in target
3. Computing similarity based on distance and angle between tangent vectors

navis includes a fast NBLAST implementation (uses compiled Rust code under the hood).

**Note:** For this tutorial, we'll demonstrate the code but not execute it since it requires additional computation time. Remove `eval=False` to run it yourself.

In [ ]:
# Convert neurons to dotprops (point clouds with tangent vectors)
# This removes connectivity but preserves spatial structure
print("Converting neurons to dotprops representation...")
neurons_dp = navis.make_dotprops(neurons_um, k=5)

print("✓ Converted to dotprops")
print(f"  {len(neurons_dp)} neurons ready for NBLAST")

# Run NBLAST all-by-all comparison
# When query and target are the same, performs all-by-all
# Use all available cores (minus 1 to keep system responsive)
import multiprocessing
n_cores = max(1, multiprocessing.cpu_count() - 1)

print(f"\nRunning NBLAST using {n_cores} cores (this may take a moment)...")
nblast_scores = navis.nblast(neurons_dp, neurons_dp, n_cores=n_cores)

print("✓ NBLAST complete!")
print(f"Score matrix shape: {nblast_scores.shape}")

# Display score matrix
nblast_scores.head()

### Hierarchical Clustering

Use NBLAST scores to cluster neurons by morphological similarity:

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from scipy.spatial.distance import squareform
import plotly.figure_factory as ff

# Convert NBLAST scores to distance matrix
# NBLAST scores are similarity scores (higher = more similar)
# We need to normalize by self-scores and convert to distances
self_scores = np.diag(nblast_scores.values)
normalized_scores = nblast_scores.values / self_scores.max()
distance_matrix = 1 - normalized_scores

# Convert to condensed distance matrix for linkage
# Only use upper triangle (distance matrix is symmetric)
condensed_dist = squareform(distance_matrix, checks=False)

# Perform hierarchical clustering
linkage_matrix = linkage(condensed_dist, method='ward')

# Choose number of clusters
k = 20
print(f"Cutting dendrogram into {k} clusters...")

# Cut tree to get cluster assignments
clusters = cut_tree(linkage_matrix, n_clusters=k).flatten() + 1  # +1 to start from 1

# Get cut height for k clusters
# Height at which we cut is at the (n-k)th merge
cut_height = linkage_matrix[len(linkage_matrix) - k, 2]

print(f"Cut height for k={k}: {cut_height:.4f}")
print(f"\nCluster sizes:")
cluster_counts = pd.Series(clusters).value_counts().sort_index()
print(cluster_counts)

# Create dendrogram with plotly
# First create using scipy to get dendrogram coordinates
dend = dendrogram(linkage_matrix, no_plot=True)

# Get leaf order and corresponding clusters
leaf_order = dend['leaves']
leaf_clusters = clusters[leaf_order]

# Get cell types for leaves (map back through leaf_order to original neurons)
leaf_cell_types = morpho_df['cell_type'].iloc[leaf_order].values

# Create color map for clusters
import plotly.express as px
colors_list = px.colors.qualitative.Light24 + px.colors.qualitative.Dark24
cluster_colors = {i: colors_list[i % len(colors_list)] for i in range(1, k+1)}
leaf_colors = [cluster_colors[c] for c in leaf_clusters]

# Create figure
fig = go.Figure()

# Add dendrogram segments
for i, (x, y) in enumerate(zip(dend['icoord'], dend['dcoord'])):
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        line=dict(color='grey', width=1),
        hoverinfo='skip',
        showlegend=False
    ))

# Add colored points at leaves with cell_type hover text
leaf_x = [(dend['icoord'][i][1] + dend['icoord'][i][2]) / 2 
          for i in range(len(dend['icoord'])) 
          if dend['dcoord'][i][0] == 0]
leaf_y = [0] * len(leaf_x)

# Group leaves by cluster for legend, including cell_type in hover text
for cluster_id in sorted(cluster_counts.index):
    cluster_mask = leaf_clusters == cluster_id
    cluster_x = [x for i, x in enumerate(leaf_x) if cluster_mask[i]]
    cluster_y = [0] * len(cluster_x)
    
    # Get cell types for this cluster's leaves
    cluster_cell_types = [leaf_cell_types[i] for i, mask in enumerate(cluster_mask) if mask]
    
    fig.add_trace(go.Scatter(
        x=cluster_x,
        y=cluster_y,
        mode='markers',
        marker=dict(size=8, color=cluster_colors[cluster_id]),
        name=f'Cluster {cluster_id}',
        hovertext=[f'Cell Type: {ct}<br>Cluster: {cluster_id}' for ct in cluster_cell_types],
        hoverinfo='text'
    ))

# Add horizontal cut line
fig.add_hline(
    y=cut_height,
    line=dict(color='red', width=2, dash='dash'),
    annotation_text=f'Cut height (k={k})',
    annotation_position='right'
)

# Update layout
fig.update_layout(
    title=dict(
        text=f'Hierarchical Clustering of Neuron Morphology (NBLAST)<br><sub>{DATASET} - {SUBSET_NAME} (k={k} clusters)</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis=dict(
        title='Neurons',
        showticklabels=False,
        showgrid=False
    ),
    yaxis=dict(
        title='Height (Ward Distance)',
        showgrid=True,
        gridcolor='lightgrey'
    ),
    plot_bgcolor='white',
    width=1200,
    height=600,
    hovermode='closest',
    legend=dict(
        orientation='v',
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.01,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='grey',
        borderwidth=1
    )
)

# Save static version
fig.write_image(f"{IMG_DIR}/{DATASET}_{SUBSET_NAME}_nblast_dendrogram.png", 
                width=1400, height=700, scale=2)
print(f"\n✓ Dendrogram saved to {IMG_DIR}/{DATASET}_{SUBSET_NAME}_nblast_dendrogram.png")

# Display interactive version
fig.show()

### Visualize Individual Clusters

Let's visualize each cluster separately to examine morphological patterns within clusters.

In [ ]:
# Visualize Cluster 1
cluster_1_neurons = [neurons_um[i] for i in range(len(neurons_um)) if clusters[i] == 1]

if len(cluster_1_neurons) > 0:
    print(f'Cluster {1}: {len(cluster_1_neurons)} neurons')
    
    # Create figure
    fig = go.Figure()
    
    # Add leg mesh if available
    if 'leg_mesh' in locals() and leg_mesh is not None:
        fig.add_trace(go.Mesh3d(
            x=leg_mesh.vertices[:, 0],
            y=leg_mesh.vertices[:, 1],
            z=leg_mesh.vertices[:, 2],
            i=leg_mesh.faces[:, 0],
            j=leg_mesh.faces[:, 1],
            k=leg_mesh.faces[:, 2],
            color='lightgrey',
            opacity=0.1,
            name='Leg Neuropil',
            hoverinfo='name'
        ))
    
    # Add neurons in cluster color
    cluster_color = cluster_colors[1]
    for i, neuron in enumerate(cluster_1_neurons):
        neuron_trace = neuron.nodes
        fig.add_trace(go.Scatter3d(
            x=neuron_trace['x'],
            y=neuron_trace['y'],
            z=neuron_trace['z'],
            mode='lines',
            line=dict(color=cluster_color, width=2),
            name=f'Neuron {i+1}',
            hoverinfo='name',
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Cluster 1 Morphology ({len(cluster_1_neurons)} neurons)',
        scene=dict(
            xaxis_title='X (µm)',
            yaxis_title='Y (µm)',
            zaxis_title='Z (µm)',
            aspectmode='data'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
else:
    print(f'No neurons in cluster 1')

In [ ]:
# Visualize Cluster 2
cluster_2_neurons = [neurons_um[i] for i in range(len(neurons_um)) if clusters[i] == 2]

if len(cluster_2_neurons) > 0:
    print(f'Cluster {2}: {len(cluster_2_neurons)} neurons')
    
    # Create figure
    fig = go.Figure()
    
    # Add leg mesh if available
    if 'leg_mesh' in locals() and leg_mesh is not None:
        fig.add_trace(go.Mesh3d(
            x=leg_mesh.vertices[:, 0],
            y=leg_mesh.vertices[:, 1],
            z=leg_mesh.vertices[:, 2],
            i=leg_mesh.faces[:, 0],
            j=leg_mesh.faces[:, 1],
            k=leg_mesh.faces[:, 2],
            color='lightgrey',
            opacity=0.1,
            name='Leg Neuropil',
            hoverinfo='name'
        ))
    
    # Add neurons in cluster color
    cluster_color = cluster_colors[2]
    for i, neuron in enumerate(cluster_2_neurons):
        neuron_trace = neuron.nodes
        fig.add_trace(go.Scatter3d(
            x=neuron_trace['x'],
            y=neuron_trace['y'],
            z=neuron_trace['z'],
            mode='lines',
            line=dict(color=cluster_color, width=2),
            name=f'Neuron {i+1}',
            hoverinfo='name',
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Cluster 2 Morphology ({len(cluster_2_neurons)} neurons)',
        scene=dict(
            xaxis_title='X (µm)',
            yaxis_title='Y (µm)',
            zaxis_title='Z (µm)',
            aspectmode='data'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
else:
    print(f'No neurons in cluster 2')

In [ ]:
# Visualize Cluster 3
cluster_3_neurons = [neurons_um[i] for i in range(len(neurons_um)) if clusters[i] == 3]

if len(cluster_3_neurons) > 0:
    print(f'Cluster {3}: {len(cluster_3_neurons)} neurons')
    
    # Create figure
    fig = go.Figure()
    
    # Add leg mesh if available
    if 'leg_mesh' in locals() and leg_mesh is not None:
        fig.add_trace(go.Mesh3d(
            x=leg_mesh.vertices[:, 0],
            y=leg_mesh.vertices[:, 1],
            z=leg_mesh.vertices[:, 2],
            i=leg_mesh.faces[:, 0],
            j=leg_mesh.faces[:, 1],
            k=leg_mesh.faces[:, 2],
            color='lightgrey',
            opacity=0.1,
            name='Leg Neuropil',
            hoverinfo='name'
        ))
    
    # Add neurons in cluster color
    cluster_color = cluster_colors[3]
    for i, neuron in enumerate(cluster_3_neurons):
        neuron_trace = neuron.nodes
        fig.add_trace(go.Scatter3d(
            x=neuron_trace['x'],
            y=neuron_trace['y'],
            z=neuron_trace['z'],
            mode='lines',
            line=dict(color=cluster_color, width=2),
            name=f'Neuron {i+1}',
            hoverinfo='name',
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Cluster 3 Morphology ({len(cluster_3_neurons)} neurons)',
        scene=dict(
            xaxis_title='X (µm)',
            yaxis_title='Y (µm)',
            zaxis_title='Z (µm)',
            aspectmode='data'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
else:
    print(f'No neurons in cluster 3')

In [ ]:
# Visualize Cluster 4
cluster_4_neurons = [neurons_um[i] for i in range(len(neurons_um)) if clusters[i] == 4]

if len(cluster_4_neurons) > 0:
    print(f'Cluster {4}: {len(cluster_4_neurons)} neurons')
    
    # Create figure
    fig = go.Figure()
    
    # Add leg mesh if available
    if 'leg_mesh' in locals() and leg_mesh is not None:
        fig.add_trace(go.Mesh3d(
            x=leg_mesh.vertices[:, 0],
            y=leg_mesh.vertices[:, 1],
            z=leg_mesh.vertices[:, 2],
            i=leg_mesh.faces[:, 0],
            j=leg_mesh.faces[:, 1],
            k=leg_mesh.faces[:, 2],
            color='lightgrey',
            opacity=0.1,
            name='Leg Neuropil',
            hoverinfo='name'
        ))
    
    # Add neurons in cluster color
    cluster_color = cluster_colors[4]
    for i, neuron in enumerate(cluster_4_neurons):
        neuron_trace = neuron.nodes
        fig.add_trace(go.Scatter3d(
            x=neuron_trace['x'],
            y=neuron_trace['y'],
            z=neuron_trace['z'],
            mode='lines',
            line=dict(color=cluster_color, width=2),
            name=f'Neuron {i+1}',
            hoverinfo='name',
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Cluster 4 Morphology ({len(cluster_4_neurons)} neurons)',
        scene=dict(
            xaxis_title='X (µm)',
            yaxis_title='Y (µm)',
            zaxis_title='Z (µm)',
            aspectmode='data'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
else:
    print(f'No neurons in cluster 4')

In [ ]:
# Visualize Cluster 5
cluster_5_neurons = [neurons_um[i] for i in range(len(neurons_um)) if clusters[i] == 5]

if len(cluster_5_neurons) > 0:
    print(f'Cluster {5}: {len(cluster_5_neurons)} neurons')
    
    # Create figure
    fig = go.Figure()
    
    # Add leg mesh if available
    if 'leg_mesh' in locals() and leg_mesh is not None:
        fig.add_trace(go.Mesh3d(
            x=leg_mesh.vertices[:, 0],
            y=leg_mesh.vertices[:, 1],
            z=leg_mesh.vertices[:, 2],
            i=leg_mesh.faces[:, 0],
            j=leg_mesh.faces[:, 1],
            k=leg_mesh.faces[:, 2],
            color='lightgrey',
            opacity=0.1,
            name='Leg Neuropil',
            hoverinfo='name'
        ))
    
    # Add neurons in cluster color
    cluster_color = cluster_colors[5]
    for i, neuron in enumerate(cluster_5_neurons):
        neuron_trace = neuron.nodes
        fig.add_trace(go.Scatter3d(
            x=neuron_trace['x'],
            y=neuron_trace['y'],
            z=neuron_trace['z'],
            mode='lines',
            line=dict(color=cluster_color, width=2),
            name=f'Neuron {i+1}',
            hoverinfo='name',
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Cluster 5 Morphology ({len(cluster_5_neurons)} neurons)',
        scene=dict(
            xaxis_title='X (µm)',
            yaxis_title='Y (µm)',
            zaxis_title='Z (µm)',
            aspectmode='data'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
else:
    print(f'No neurons in cluster 5')

In [ ]:
# Visualize Cluster 6
cluster_6_neurons = [neurons_um[i] for i in range(len(neurons_um)) if clusters[i] == 6]

if len(cluster_6_neurons) > 0:
    print(f'Cluster {6}: {len(cluster_6_neurons)} neurons')
    
    # Create figure
    fig = go.Figure()
    
    # Add leg mesh if available
    if 'leg_mesh' in locals() and leg_mesh is not None:
        fig.add_trace(go.Mesh3d(
            x=leg_mesh.vertices[:, 0],
            y=leg_mesh.vertices[:, 1],
            z=leg_mesh.vertices[:, 2],
            i=leg_mesh.faces[:, 0],
            j=leg_mesh.faces[:, 1],
            k=leg_mesh.faces[:, 2],
            color='lightgrey',
            opacity=0.1,
            name='Leg Neuropil',
            hoverinfo='name'
        ))
    
    # Add neurons in cluster color
    cluster_color = cluster_colors[6]
    for i, neuron in enumerate(cluster_6_neurons):
        neuron_trace = neuron.nodes
        fig.add_trace(go.Scatter3d(
            x=neuron_trace['x'],
            y=neuron_trace['y'],
            z=neuron_trace['z'],
            mode='lines',
            line=dict(color=cluster_color, width=2),
            name=f'Neuron {i+1}',
            hoverinfo='name',
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Cluster 6 Morphology ({len(cluster_6_neurons)} neurons)',
        scene=dict(
            xaxis_title='X (µm)',
            yaxis_title='Y (µm)',
            zaxis_title='Z (µm)',
            aspectmode='data'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
else:
    print(f'No neurons in cluster 6')

In [ ]:
# Visualize Cluster 7
cluster_7_neurons = [neurons_um[i] for i in range(len(neurons_um)) if clusters[i] == 7]

if len(cluster_7_neurons) > 0:
    print(f'Cluster {7}: {len(cluster_7_neurons)} neurons')
    
    # Create figure
    fig = go.Figure()
    
    # Add leg mesh if available
    if 'leg_mesh' in locals() and leg_mesh is not None:
        fig.add_trace(go.Mesh3d(
            x=leg_mesh.vertices[:, 0],
            y=leg_mesh.vertices[:, 1],
            z=leg_mesh.vertices[:, 2],
            i=leg_mesh.faces[:, 0],
            j=leg_mesh.faces[:, 1],
            k=leg_mesh.faces[:, 2],
            color='lightgrey',
            opacity=0.1,
            name='Leg Neuropil',
            hoverinfo='name'
        ))
    
    # Add neurons in cluster color
    cluster_color = cluster_colors[7]
    for i, neuron in enumerate(cluster_7_neurons):
        neuron_trace = neuron.nodes
        fig.add_trace(go.Scatter3d(
            x=neuron_trace['x'],
            y=neuron_trace['y'],
            z=neuron_trace['z'],
            mode='lines',
            line=dict(color=cluster_color, width=2),
            name=f'Neuron {i+1}',
            hoverinfo='name',
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Cluster 7 Morphology ({len(cluster_7_neurons)} neurons)',
        scene=dict(
            xaxis_title='X (µm)',
            yaxis_title='Y (µm)',
            zaxis_title='Z (µm)',
            aspectmode='data'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
else:
    print(f'No neurons in cluster 7')

In [ ]:
# Visualize Cluster 8
cluster_8_neurons = [neurons_um[i] for i in range(len(neurons_um)) if clusters[i] == 8]

if len(cluster_8_neurons) > 0:
    print(f'Cluster {8}: {len(cluster_8_neurons)} neurons')
    
    # Create figure
    fig = go.Figure()
    
    # Add leg mesh if available
    if 'leg_mesh' in locals() and leg_mesh is not None:
        fig.add_trace(go.Mesh3d(
            x=leg_mesh.vertices[:, 0],
            y=leg_mesh.vertices[:, 1],
            z=leg_mesh.vertices[:, 2],
            i=leg_mesh.faces[:, 0],
            j=leg_mesh.faces[:, 1],
            k=leg_mesh.faces[:, 2],
            color='lightgrey',
            opacity=0.1,
            name='Leg Neuropil',
            hoverinfo='name'
        ))
    
    # Add neurons in cluster color
    cluster_color = cluster_colors[8]
    for i, neuron in enumerate(cluster_8_neurons):
        neuron_trace = neuron.nodes
        fig.add_trace(go.Scatter3d(
            x=neuron_trace['x'],
            y=neuron_trace['y'],
            z=neuron_trace['z'],
            mode='lines',
            line=dict(color=cluster_color, width=2),
            name=f'Neuron {i+1}',
            hoverinfo='name',
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Cluster 8 Morphology ({len(cluster_8_neurons)} neurons)',
        scene=dict(
            xaxis_title='X (µm)',
            yaxis_title='Y (µm)',
            zaxis_title='Z (µm)',
            aspectmode='data'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
else:
    print(f'No neurons in cluster 8')

---

## Template Brain Transformations

The [**flybrains**](https://github.com/navis-org/navis-flybrains) package provides template brain transformations for Drosophila connectomes.

It supports 31+ template brains including:
- FAFB14, FLYWIRE
- JRC2018F/M/U (Janelia Reference Brains)
- JRCFIB2022M (maleCNS)
- FANC, BANC, MANC
- VNC templates

**Note:** This requires external dependencies (CMTK or Elastix). See [flybrains documentation](https://github.com/navis-org/navis-flybrains) for installation.

In [ ]:
try:
    import flybrains
    
    print("✓ flybrains loaded")
    print(f"  Version: {flybrains.__version__}")
    
    # Download optional transforms (only needs to be done once)
    # Uncomment these if you haven't downloaded them yet:
    # flybrains.download_jefferislab_transforms()
    # flybrains.download_jrc_transforms()
    # flybrains.download_jrc_vnc_transforms()
    # flybrains.register_transforms()
    
    # Check available transforms
    print("\nExample: Transform BANC neurons to JRC2018F space:")
    print("")
    print("```python")
    print("# Transform a single neuron")
    print("neuron_jrc = navis.xform_brain(neuron_um, source='BANC', target='JRC2018F')")
    print("")
    print("# Transform multiple neurons")
    print("neurons_jrc = navis.xform_brain(neurons_um, source='BANC', target='JRC2018F')")
    print("```")
    
except ImportError:
    print("flybrains not installed")
    print("Install with: pip install flybrains")
    print("See: https://github.com/navis-org/navis-flybrains")

### Transform MANC Neurons to BANC Space

While we provide pre-transformed skeletons, you can also perform transformations yourself using **flybrains**.

This is useful when:
- Working with custom neurons not in our dataset
- Transforming synapses or other 3D data
- Needing intermediate template spaces

Here's how to transform MANC neurons from native space to BANC space:

<p align="center">
  <img src="../inst/images/bridging_graph.png" alt="Template Brain Bridging Graph" width="80%">
</p>

**Note:** This code cell is set to not execute automatically (just for demonstration). To run it yourself:
1. Install flybrains: `pip install flybrains`
2. Download transforms (one-time setup, see code comments)
3. Remove the `raise` statement and execute the cell

In [ ]:
# NOTE: This cell is for demonstration only - it will not execute automatically
# Remove the raise statement below to run this example
raise NotImplementedError(
    "This is a demonstration cell. To execute:\n"
    "1. Install flybrains: pip install flybrains\n"
    "2. Download transforms (see comments below)\n"
    "3. Remove this raise statement and re-run"
)

# One-time setup: Download transformation registrations
# Uncomment and run once:
# import flybrains
# flybrains.download_jefferislab_transforms()
# flybrains.download_jrc_transforms()
# flybrains.download_jrc_vnc_transforms()
# flybrains.register_transforms()

# Switch to MANC dataset
dataset_manc = "manc_121"
dataset_manc_id = "manc_121_id"

# Read MANC meta data
meta_manc_path = construct_path(DATA_PATH, dataset_manc, "meta")
meta_manc_full = read_feather_gcs(meta_manc_path, gcs_fs=gcs)

# Filter for DNg12 neurons in MANC
meta_manc_dng12 = meta_manc_full[
    meta_manc_full['cell_type'].str.contains('DNg12', case=False, na=False)
]
print(f"Found {len(meta_manc_dng12)} DNg12 neurons in MANC")

# Construct path to MANC skeletons in NATIVE MANC space
skeletons_manc_path = construct_path(
    DATA_PATH,
    dataset_manc,
    "skeletons",
    space_suffix=f"{dataset_manc}_manc_space"
)

# Read MANC neurons in native space
manc_ids = meta_manc_dng12[dataset_manc_id].values
manc_filenames = [f"{nid}.swc" for nid in manc_ids]

if USE_GCS:
    gcs_manc_path = skeletons_manc_path.replace("gs://", "")
    manc_dng12 = batch_read_swc_from_gcs(
        gcs,
        gcs_manc_path,
        manc_filenames,
        show_progress=False
    )
else:
    manc_dng12 = navis.NeuronList([
        navis.read_swc(f"{skeletons_manc_path}/{fname}")
        for fname in manc_filenames
    ])

print(f"Loaded {len(manc_dng12)} MANC neurons in native space")

# Transform MANC neurons to JRCVNC2018F template
print("Transforming MANC -> JRCVNC2018F...")
manc_dng12_jrcvnc2018f = navis.xform_brain(
    manc_dng12,
    source='MANC',
    target='JRCVNC2018F'
)

# Transform from JRCVNC2018F to BANC
# Note: For this step, you would need bancr-equivalent functionality in Python
# This is currently only available in R via bancr::banc_to_JRC2018F()
# For now, use the pre-transformed skeletons in BANC space
print("\nNote: Direct JRCVNC2018F -> BANC transform requires additional setup.")
print("Alternative: Use pre-transformed skeletons with space_suffix='banc_space'")

# Visualize the intermediate transform
manc_dng12_jrcvnc2018f_um = manc_dng12_jrcvnc2018f / 1000

fig = navis.plot3d(
    manc_dng12_jrcvnc2018f_um,
    backend='plotly',
    color='green',
    width=1200,
    height=800,
    title="MANC DNg12 Neurons in JRCVNC2018F Space"
)

fig.show()

print("\nTransformation notes:")
print("- MANC -> JRCVNC2018F: Available via flybrains")
print("- JRCVNC2018F -> BANC: Requires additional bridging registration")
print("- For production use: Use our pre-transformed skeletons in BANC space")

---

## Summary

In this tutorial, we covered:

1. **Loading Neuron Skeletons**: Reading SWC files from GCS with navis
2. **3D Visualization**: Interactive plotting with plotly backend
3. **Morphometric Analysis**: Extracting cable length, branch counts, etc.
4. **Neuropil Meshes**: Visualizing neurons in anatomical context
5. **NBLAST**: Morphological similarity comparison
6. **Template Brains**: Introduction to flybrains for spatial transformations

### Key navis Features

- **Simple API**: `navis.read_swc()`, `navis.plot3d()`, `navis.nblast()`
- **Fast**: Compiled code (Rust) for NBLAST and other operations
- **Flexible**: Multiple backends (plotly, octarine, k3d)
- **Integrated**: Works with pandas, numpy, and other scientific Python tools
- **Well-documented**: Extensive [documentation](https://navis.readthedocs.io/) and [tutorials](https://navis-org.github.io/navis/)

### Useful navis Functions

```python
# Reading neurons
navis.read_swc('neuron.swc')
navis.read_swc('neurons.zip', include_subdirs=True)

# Morphometrics
n.cable_length  # Total cable length
n.n_branches    # Number of branches
n.n_nodes       # Number of nodes
navis.segment_analysis(n)  # Detailed per-segment metrics

# Visualization
navis.plot3d(neurons, backend='plotly', color='blue')
navis.plot3d(neurons, backend='octarine')  # Modern, fast backend

# Morphological comparison
dotprops = navis.make_dotprops(neurons, k=5)
scores = navis.nblast(dotprops, dotprops)

# Spatial transformations
navis.xform_brain(neuron, source='BANC', target='JRC2018F')

# Manipulation
neurons / 1000  # Scale (e.g., nm to µm)
navis.downsample_neuron(n, factor=5)  # Reduce resolution
navis.prune_by_strahler(n, to_prune=1)  # Prune terminal branches
```

---

## Next Steps

- Explore Tutorial 03 for connectivity analysis
- Try different datasets (FAFB, MANC, etc.)
- Experiment with different NBLAST parameters
- Use flybrains to compare neurons across datasets
- Check out the [navis documentation](https://navis.readthedocs.io/) for more advanced features

---

**Tutorial complete!** 🎉

## Session Information

In [ ]:
import sys
import platform

print("Python Session Information")
print("=" * 50)
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print("\nPackage Versions:")
print(f"  navis: {navis.__version__}")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")

try:
    import flybrains
    print(f"  flybrains: {flybrains.__version__}")
except:
    print(f"  flybrains: not installed")

print("\n" + "=" * 50)